In [1]:
import login
import psycopg2
try:
    conn = psycopg2.connect(user = login.user,
                                  password = login.password,
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "cookingapp")

    cur = conn.cursor()
    print ( conn.get_dsn_parameters(),"\n")
    cur.execute("SELECT version();")
    record = cur.fetchone()
    print("You are connected to - ", record,"\n")
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'cook', 'dbname': 'cookingapp', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 12.0, compiled by Visual C++ build 1914, 64-bit',) 



In [2]:
import pandas as pd
import numpy as np

In [3]:
zvolene_kategorie = ["Vegetarian"]
zvolene_ingredience = ["butter", "bacon"]

sql1 = "INSERT INTO zvolene_kategorie(idk) VALUES ((SELECT kid FROM kategorie WHERE nazev = %s))"
sql2 = "INSERT INTO zvolene_ingredience(idi) VALUES ((SELECT iid FROM ingredience_pref WHERE nazev = %s))"

try:
    cur.execute("DELETE FROM zvolene_kategorie")
    cur.execute("DELETE FROM zvolene_ingredience")
    cur.executemany(sql1, [[x] for x in zvolene_kategorie])
    cur.executemany(sql2, [[x] for x in zvolene_ingredience])
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
    conn.rollback()

In [6]:
sql = ("SELECT * from navrzene_recepty ORDER BY "
"(CAST(shoda AS DOUBLE PRECISION)/pocet_ingredienci*CAST(shoda AS DOUBLE PRECISION)/zvoleno_ingredienci) DESC")

df = pd.read_sql(sql, conn)
df.head()

,rid,shoda,zvoleno_ingredienci,pocet_ingredienci,calories,protein,rating,sodium,fat,title,description,date
0,17732,1,2,1,203.0,0.0,5.000,3.0,23.0,Indian Clarified Butter,South Indians don't use ghee in cooking the wa...,2012-02-15 04:00:00
1,1445,1,2,1,203.0,0.0,5.000,3.0,23.0,Indian Clarified Butter,South Indians don't use ghee in cooking the wa...,2012-02-15 04:00:00
2,3518,1,2,2,NaN,NaN,4.375,NaN,NaN,Jelly Bowl Cookies,None,2004-08-20 12:48:48
3,5678,1,2,2,169.0,5.0,4.375,5.0,6.0,Fresh Pumpkin Purée,Active time: 25 min Start to finish: 10 1/2 hr,2004-08-20 04:00:00
4,11234,1,2,2,41.0,0.0,4.375,1.0,0.0,Classic Shortbread,Editor's note: This recipe is from Ming Tsai's...,2006-11-08 20:36:33


In [7]:
cur.close()
conn.close()